In [40]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [63]:
data = pd.read_csv("train.csv") 

In [64]:
data.loc[(data["Cabin"].isna()),"Cabin"] = 0
data.loc[(data["Cabin"]!=0),"Cabin"] = 1

In [65]:
data = data.rename(columns = {"Cabin":"HasCabin"})

In [66]:
sex_mean_age = data.groupby("Sex").aggregate({"Age":"mean"})
sex_mean_age = sex_mean_age.transpose()
data["Sex"] = data["Sex"].apply(lambda x : int(x=="male"))
data.loc[(data["Sex"] == True)&(data["Age"].isna()),"Age"] = round(sex_mean_age.male.iloc[0])
data.loc[(data["Sex"] == False)&(data["Age"].isna()),"Age"] = round(sex_mean_age.female.iloc[0])

In [67]:
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
data['Embarked'] = data['Embarked'].map({"S":-1,"C":0,"Q":1})

In [68]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,HasCabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,0,-1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,1,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,0,-1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,1,-1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,0,-1
5,6,0,3,"Moran, Mr. James",1,31.0,0,0,330877,8.4583,0,1
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.0,0,0,17463,51.8625,1,-1
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.0,3,1,349909,21.0750,0,-1
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.0,0,2,347742,11.1333,0,-1
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.0,1,0,237736,30.0708,0,0


In [69]:
X = data.drop("Survived",axis=1)
y = data.Survived

In [70]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [71]:
X = X.drop(["Name","Ticket"],axis=1)
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.25)

In [72]:
X_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,HasCabin,Embarked
335,336,3,1,31.00,0,0,7.8958,0,-1
620,621,3,1,27.00,1,0,14.4542,0,0
240,241,3,0,28.00,1,0,14.4542,0,0
76,77,3,1,31.00,0,0,7.8958,0,-1
469,470,3,0,0.75,2,1,19.2583,0,0


## Tree

In [73]:
from sklearn.tree import DecisionTreeClassifier

In [74]:
tree = DecisionTreeClassifier()
param_grid={"criterion":["gini","entropy"],"max_depth":range(1,10)}
search = GridSearchCV(tree, param_grid=param_grid, cv=5)

In [75]:
search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 10)})

In [76]:
best_tree = search.best_estimator_

In [77]:
from sklearn.metrics import f1_score 

In [78]:

y_val_pred = best_tree.predict(X_val)
print("F1 Score на валидационной выборке:", f1_score(y_val, y_val_pred))

F1 Score на валидационной выборке: 0.7654320987654321


## KNN

In [99]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn_param_grid={"n_neighbors":range(1,50)}
knn_search = GridSearchCV(knn, param_grid=knn_param_grid, cv=5)

In [102]:
knn_search.fit(X_train[["Age","Sex"]],y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 50)})

In [103]:
best_knn = knn_search.best_estimator_

In [104]:

y_val_pred = best_knn.predict(X_val[["Age","Sex"]])
print("F1 Score на валидационной выборке:", f1_score(y_val, y_val_pred))

F1 Score на валидационной выборке: 0.5648854961832062
